Imports + DB connection

In [1]:
from pathlib import Path
import sqlite3
import pandas as pd

# Project root = one level above scripts/
ROOT = Path.cwd().parents[1]

DATA_DIR = ROOT / "data"
DB_PATH  = DATA_DIR / "econ_dev.db"

print("Project root:", ROOT)
print("Data dir:", DATA_DIR)
print("DB path:", DB_PATH)

conn = sqlite3.connect(DB_PATH)
conn.execute("PRAGMA foreign_keys = ON;")


Project root: /Users/lauraz/final-project-group1
Data dir: /Users/lauraz/final-project-group1/data
DB path: /Users/lauraz/final-project-group1/data/econ_dev.db


Import and merge WDI indicator files

In [2]:
# This block reads the three raw World Bank indicator files, reshapes them into a country–year format,
# and merges all indicators into a single database table for analysis.

csv_files = [
    "wdi_employment_raw.csv",       
    "wdi_gdp_growth_raw.csv",  
    "wdi_gdp_per_capita_raw.csv"  
]

# Start fresh by dropping existing wdi_raw table if it exists
conn.execute("DROP TABLE IF EXISTS wdi_raw;")
conn.commit()

# Loop through each file and process
for fname in csv_files:
    path = DATA_DIR / fname
    print("Loading:", path)
    
    # Read WDI file, skipping the 4 metadata rows
    df = pd.read_csv(path, skiprows=4)
    
    # Reshape from wide (many year columns) to long (Year, Value)
    id_vars = ["Country Name", "Country Code", "Indicator Name", "Indicator Code"]
    df_long = df.melt(
        id_vars=id_vars,
        var_name="Year",
        value_name="Value"
    )
    
    # Clean up column names
    df_long = df_long.rename(columns={
        "Country Name":   "country_name",
        "Country Code":   "country_code",
        "Indicator Name": "indicator_name",
        "Indicator Code": "indicator_code",
        "Year":           "year",
        "Value":          "value"
    })
    
    # Make Year an integer
    df_long["year"] = pd.to_numeric(df_long["year"], errors="coerce")
    
    # Drop rows where year is NA in case of bad data
    df_long = df_long.dropna(subset=["year"])
    df_long["year"] = df_long["year"].astype(int)
    
    # Append to wdi_raw in the database
    df_long.to_sql("wdi_raw", conn, if_exists="append", index=False)

print("Done importing all files.")


Loading: /Users/lauraz/final-project-group1/data/wdi_employment_raw.csv
Loading: /Users/lauraz/final-project-group1/data/wdi_gdp_growth_raw.csv
Loading: /Users/lauraz/final-project-group1/data/wdi_gdp_per_capita_raw.csv
Done importing all files.


In [5]:
# This block reorganizes the combined data by separating country information, indicator descriptions,
# and yearly values into distinct tables to reduce duplication and simplify later analysis.

schema_sql = """
DROP TABLE IF EXISTS observations;
DROP TABLE IF EXISTS countries;
DROP TABLE IF EXISTS indicators;

CREATE TABLE countries (
    country_code TEXT PRIMARY KEY,
    country_name TEXT
);

CREATE TABLE indicators (
    indicator_code TEXT PRIMARY KEY,
    indicator_name TEXT,
    topic TEXT
);

CREATE TABLE observations (
    country_code   TEXT,
    indicator_code TEXT,
    year           INTEGER,
    value          REAL,
    PRIMARY KEY (country_code, indicator_code, year)
);

INSERT INTO countries (country_code, country_name)
SELECT DISTINCT country_code, country_name
FROM wdi_raw;

INSERT INTO indicators (indicator_code, indicator_name, topic)
SELECT DISTINCT indicator_code, indicator_name, 'Economic Development'
FROM wdi_raw;

INSERT INTO observations (country_code, indicator_code, year, value)
SELECT country_code, indicator_code, year, value
FROM wdi_raw;
"""

conn.executescript(schema_sql)
conn.commit()

print("Countries, indicators, and observations tables created.")


Countries, indicators, and observations tables created.


In [7]:
# This block imports World Bank income group classifications for each country.

income_path = DATA_DIR / "wdi_income_group.csv"

income_df = pd.read_csv(income_path)

income_df = income_df.rename(columns={
    "Country Code": "country_code",
    "IncomeGroup": "income_group"
})

income_df = income_df[["country_code", "income_group"]]

conn.execute("DROP TABLE IF EXISTS country_income;")
conn.commit()

income_df.to_sql("country_income", conn, index=False)

print("country_income table created.")


country_income table created.


In [8]:
# This block filters the dataset to a consistent time period while retaining all countries
# for income-group-based analysis.

clean_sql = """
DROP TABLE IF EXISTS observations_clean;

CREATE TABLE observations_clean AS
SELECT *
FROM observations
WHERE year BETWEEN 2000 AND 2023;
"""

conn.executescript(clean_sql)
conn.commit()

print("observations_clean created.")


observations_clean created.


In [9]:
# This block creates a country–year dataset with each economic indicator in its own column,
# which serves as the base table for income-group analysis.

panel_sql = """
DROP VIEW IF EXISTS panel_wide;

CREATE VIEW panel_wide AS
SELECT
    o.country_code,
    c.country_name,
    o.year,
    MAX(CASE WHEN o.indicator_code = 'SL.EMP.TOTL.SP.ZS'    THEN o.value END) AS emp_ratio,    -- employment
    MAX(CASE WHEN o.indicator_code = 'NY.GDP.MKTP.KD.ZG'    THEN o.value END) AS gdp_growth,   -- GDP growth
    MAX(CASE WHEN o.indicator_code = 'NY.GDP.PCAP.KD'       THEN o.value END) AS gdp_pc        -- GDP per capita
FROM observations_clean o
JOIN countries c USING (country_code)
GROUP BY o.country_code, c.country_name, o.year;
"""

conn.executescript(panel_sql)
conn.commit()

print("panel_wide view created.")


panel_wide view created.


In [10]:
# This block summarizes the average level of each indicator by income group across all years in the analysis window.

summary_by_income = pd.read_sql("""
    SELECT
        c.income_group,
        COUNT(*)                  AS n_country_years,
        ROUND(AVG(p.emp_ratio), 2)   AS avg_emp_ratio,
        ROUND(AVG(p.gdp_growth), 2)  AS avg_gdp_growth,
        ROUND(AVG(p.gdp_pc), 2)      AS avg_gdp_pc
    FROM panel_wide p
    JOIN country_income c USING (country_code)
    WHERE c.income_group IS NOT NULL
    GROUP BY c.income_group
    ORDER BY avg_gdp_pc DESC;
""", conn)

summary_by_income


,income_group,n_country_years,avg_emp_ratio,avg_gdp_growth,avg_gdp_pc
0,High income,2064,57.22,2.54,35295.24
1,Upper middle income,1296,53.23,3.57,5815.63
2,Lower middle income,1200,55.21,4.20,2030.59
3,Low income,600,61.35,3.77,703.20


In [11]:
# This block computes yearly averages of each indicator by income group to examine how development patterns evolve over time.

summary_by_income_year = pd.read_sql("""
    SELECT
        c.income_group,
        p.year,
        ROUND(AVG(p.emp_ratio), 2)   AS avg_emp_ratio,
        ROUND(AVG(p.gdp_growth), 2)  AS avg_gdp_growth,
        ROUND(AVG(p.gdp_pc), 2)      AS avg_gdp_pc
    FROM panel_wide p
    JOIN country_income c USING (country_code)
    WHERE c.income_group IS NOT NULL
    GROUP BY c.income_group, p.year
    ORDER BY c.income_group, p.year;
""", conn)

summary_by_income_year.head()


,income_group,year,avg_emp_ratio,avg_gdp_growth,avg_gdp_pc
0,High income,2000,56.10,4.85,29733.48
1,High income,2001,56.18,2.40,30204.97
2,High income,2002,56.03,2.36,30339.30
3,High income,2003,55.97,3.08,30894.56
4,High income,2004,56.21,4.83,32013.04
